# Проект по SQL

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Поставленные задачи

1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Анализ данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Запрос для получения списка всех таблиц:

In [5]:
query_tables = '''
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
'''
result_tables = pd.read_sql(query_tables, con=con)
display(result_tables)

,table_name
0,advertisment_costs
1,author
2,authors
3,books
4,orders
5,publishers
6,ratings
7,report_vendor
8,reportt_vendor
9,reporttt_vendor


Запрос для получения столбцов таблицы ratings:

In [6]:
query_columns_ratings = '''
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'ratings';
'''
result_columns_ratings = pd.read_sql(query_columns_ratings, con=con)
display(result_columns_ratings)

,column_name
0,rating_id
1,book_id
2,rating
3,username


Запрос для получения столбцов таблицы reviews:

In [7]:
query_columns = '''
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'reviews';
'''
result_columns = pd.read_sql(query_columns, con=con)
display(result_columns)

,column_name
0,review_id
1,book_id
2,username
3,text


In [8]:
# Функция для выполнения SQL-запросов
def execute_query(query):
    with engine.connect() as connection:
        result = pd.read_sql(sql=text(query), con=connection)
    return result

In [9]:
query_columns = '''
SELECT COUNT(DISTINCT review_id) AS unique_reviews_count
FROM reviews;
'''
result_columns = execute_query(query_columns)
print("\nКоличество уникальных review_id:")
display(result_columns)


Количество уникальных review_id:


,unique_reviews_count
0,2793


### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [10]:
query_1 = '''
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
'''
result_1 = execute_query(query_1)
print("Книг после 1 января 2000 года:")
display(result_1)

Книг после 1 января 2000 года:


,books_after_2000
0,819


### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [11]:
query2 = """
SELECT 
    b.title,
    COUNT(DISTINCT r.review_id) AS num_reviews,
    AVG(rt.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY num_reviews DESC;
"""
result2 = pd.read_sql_query(query2, con)
print("Количество обзоров и средняя оценка для каждой книги (отсортировано по количеству обзоров):")
display(result2)

Количество обзоров и средняя оценка для каждой книги (отсортировано по количеству обзоров):


,title,num_reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


### Издательство с наибольшим числом книг толще 50 страниц

In [12]:
query_3 = '''
SELECT 
    p.publisher_id,
    p.publisher AS publisher_name, -- Используем корректное имя столбца
    COUNT(b.book_id) AS book_count
FROM books AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY book_count DESC
LIMIT 1;
'''
result_3 = execute_query(query_3)
print("\nИздательство с наибольшим числом книг толще 50 страниц:")
display(result_3)


Издательство с наибольшим числом книг толще 50 страниц:


,publisher_id,publisher_name,book_count
0,212,Penguin Books,42


### Автор с самой высокой средней оценкой книг (учитывая только книги с 50+ оценками)

In [13]:
query_4 = '''
WITH filtered_books AS (
    SELECT 
        b.book_id,
        b.author_id,
        AVG(rt.rating) AS avg_book_rating
    FROM books AS b
    JOIN ratings AS rt ON b.book_id = rt.book_id
    GROUP BY b.book_id, b.author_id
    HAVING COUNT(rt.rating) >= 50
),
author_ratings AS (
    SELECT 
        fb.author_id,
        AVG(fb.avg_book_rating) AS avg_author_rating
    FROM filtered_books AS fb
    GROUP BY fb.author_id
)
SELECT 
    a.author_id,
    a.author AS author_name,
    ar.avg_author_rating
FROM author_ratings AS ar
JOIN authors AS a ON ar.author_id = a.author_id
ORDER BY ar.avg_author_rating DESC
LIMIT 1;
'''
result_4 = execute_query(query_4)
print("\nАвтор с наивысшим средним рейтингом (учитывая книги с ≥ 50 оценками):")
display(result_4)


Автор с наивысшим средним рейтингом (учитывая книги с ≥ 50 оценками):


,author_id,author_name,avg_author_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


### Среднее количество обзоров от пользователей, которые оставили больше 48 оценок

In [17]:
query5 = """
WITH user_rating_counts AS (
    SELECT 
        username,
        COUNT(*) AS num_ratings
    FROM ratings
    GROUP BY username
    HAVING COUNT(*) > 48
)
SELECT 
    AVG(user_reviews.num_reviews) AS avg_reviews_per_user
FROM (
    SELECT 
        r.username,
        COUNT(*) AS num_reviews
    FROM reviews AS r
    JOIN user_rating_counts AS urc ON r.username = urc.username
    GROUP BY r.username
) AS user_reviews;
"""
result5 = pd.read_sql_query(query5, con)
print("Среднее количество обзоров от пользователей, которые поставили больше 48 оценок:")
display(result5)

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок:


,avg_reviews_per_user
0,24.0


## Вывод:

1. Число книг, выпущенных после 1 января 2000 года: 
* 819 книг


2. Количество обзоров и средняя оценка для каждой книги: 
* Например, книга "Twilight (Twilight #1)" имеет 7 обзоров и среднюю оценку 3.66.


3. Издательство с наибольшим числом книг толщиной более 50 страниц: 
* Мы определили, что издательство Penguin Books выпустило наибольшее количество книг (42 книги), которые превышают 50 страниц, исключая брошюры.


4. Автор с самой высокой средней оценкой книг: 
* По результатам анализа, автор J.K. Rowling/Mary GrandPré имеет наибольшее количество обзоров для своих книг.


5. Среднее количество обзоров от пользователей, оставивших больше 48 оценок: 
*  Среднее количество оценок составляет 24.